## Table of Contents

-----

- [Introduction](#introduction)
  - [System Structure](#system-structure)
- [Pong Game](#pong-game)
  - [Pong Heartbeat](#pong-heartbeat)
  - [Pong Buttons](#pong-buttons)
  - [Pong Initialization](#pong-initialization)
- [Menu System](#menu-system)
- [Heartbeat Events](#heartbeat-events)
- [Global Variables](#global-variables--variable-sharing)
- [Buttons and Events](#buttons-and-events)
  - [Event Handling](#event-handling)
    - [Buttons](#buttons)
      - [Ports / Gold Icon](#ports--gold-icon)
    - [Event Handler](#event-handler)

---
  
Full Code: https://github.com/ionwannabehere/Microbits/blob/main/main.py



## Introduction

### System Structure
The type of system that you use for these type of process enumeration, and several process handling is important. As well as your event handling / input handling; since the way that code is structured each line is consecutive, and becomes a struggle when wanting several things to happen at once.
The structure of my system is as follows:  

EventList ["Event 1", "Event 2", "Event 3"]  

```
Events:
    While True:
        If event is active:
            If event isnt already active:
                Add to Event List  
```
Processes {
    Menu,
    Pong,
    Settings
}
```
Process Structure:  
    ProcessInit():  
        Clear Display  
        Display Normal process  
        Set variables  
    ProcessButtonA():  
        Grab global variables  
        Change and process variables  
        Display changes  
        Set variables back  

        
Event Handling Structure:  
    When I get event  
        If Process is Active:  
            ProcessButtonA()  

Menu Structure:  
    If process is active, and gold is touched:  
        All processes off  
        Wait for completion (100ms)  
        Display Clear  
        MenuInit()  
    If Menu is Active, and gold is touched:  
        Menu off  
        Process On  
        Wait for completion (100ms)  
        Display clear  
        ProcessInit()  
```        

## Pong Game

### Pong Heartbeat

### Pong Buttons

### Pong Initialization
Each Process in this code structure is given a process Initialization function, this is required since these are not in any loop and would be laggy and bloated if conditions were in the loop.  

In [ ]:
def pongI():
    setattr(hrtValues,"H",0)
    
    r1 = random.randint(1,2)
    r2 = random.randint(1,2)
    ball = [r1,r2]
    if random.choice([True, False]):
        ballDir = -1
    else:
        ballDir = 1
    ballRef = 1
    display.set_pixel(r1,r2,5)
    
    plrD = 2
    display.set_pixel(0,2,9)
    display.set_pixel(0,3,9)
    cmpD = 2
    display.set_pixel(4,2,9)
    display.set_pixel(4,3,9)
    setattr(globalValues,"ball",ball)
    setattr(globalValues,"ballDir",ballDir)
    setattr(globalValues,"ballRef",ballRef)
    setattr(globalValues,"plrD",plrD)
    setattr(globalValues,"cmpD",cmpD)

## Menu System

## Heartbeat Events


Hearbeat events utilize the built in tick time, in order to find the difference in time frames to have the next hearbeat event to be fired. The syntax in question is `time.tick_ms()`, I made a function that will check if the next heartbeat function should be fired; this is called within the while loop at the end of the code.

In [ ]:
pongA = True

class hrtValues:
    pass

def hrtFnCh(n,z):
    if math.trunc((time.ticks_ms()/1000)*z) > getattr(hrtValues,n):
        setattr(hrtValues,n,math.trunc((time.ticks_ms()/1000)*z))
        eventHandList.append(n)

def pongI():
    setattr(hrtValues,"H",0)

while True:
    if pongA:
        hrtFnCh("H",3)

Firstly inorder to track the time difference between each event we save the heartbeat values inside of the class `hrtValues`. The function `hrtFnCh(n,z)` (or Heartbeat Function Check), is what checks if the heartbeat should be fired again, I am going to break it down here.

The condtion `math.trunc((time.ticks_ms()/1000)*z) > getattr(hrtValues,n)`, does a few things, `time.ticks_ms()` returns the amount of time in milliseconds since the device has been booted: 

`5623`

Firstly we divide by 1000 in order to break it into seconds, 

`5.623`

This is pretty sufficent but incase we want to have the heartbeat function be greater than or less than 1 second but still in regards, like x0.5 (half second) x2 (2 seconds), we multiply this by the z componet [in our case 3 for "H"], 

`16.869`

Now understand that since we are multiplying these together it is the opposite, 2 = 1/2 of a second, and 0.5 = 2 seconds. I could have fixed this inconvenience but since I was the only one using this code, and fixing would take more space I left it as is. Continuing we have `16.869`, we need to get rid of the decimals we use `math.trunc()` this rounds down the number towards 0, leaving us with 

`16`

Finaly we take this number `16`, and if it is greater than `getattr(hrtValues,n)` (or the hearbeat value stored in hrtValues, our case it is `"H"`). and if so it will set the global value to the new heartbeat number, and lastly it will append the event name to the Event Hand List, to be processed.

The rest of the code is self-explanatory, PongA is the boolean variable that is active when the process/game is active, the itialization of the game sets the global hearbeat value of `"H"` to 0, incase it was started from boot. And inside the while true loop, if pong is active, then it will run the heart function check; which checks if the hearbeat should be fired, and adds to the event list automatically due to the function.

## Global Variables / Variable Sharing


Our "global" variables take 2 shapes. The way that functions are made in python, any variables stated before the function are known as global variables in our case we have `vol = 0`. it becomes apparent that we use it as a global variable since inside our setting function for the volume we have to state `global vol` this allows the function to grab this variable. Another way we use variables and variable sharing between functions is through classes, I used `class globalValues`, this isnt entirely required since if needed you can just state all your functions at the top of the code. I use classes in this case to have all the game variables in one place for better management.

For this reason at the top of each button or input function we have to state `var = getattr(globalValues,"var")`
this does 2 things, it grabs the variable var from globalValues.
```
class globalValues
    var = 10  -----> returns
    bar = 30
    foo = 50
```
And sets this to our local variable, `var`, and we then use this variable and adjust it to our needs and at the end of the code, if needed we send it back to the class:

`setattr(globalValues,"tabA",tabA)`

There is a slight issue with this structure, since we initalize the variable, change it, and send it out, it adds a lot more code and questions the size of the code as a whole. However it also gives a pro, since we decide if we want to return the variable back or not, this gives us the extra mobility of having two thing being calculated consequtively and sent off as one. 

`[Calculate A] + [Calculate B] -> Display [A + B]`  

Compared to the conventional

[`Calculate A] -> Display [A], [Calculate B] -> Display [B]`.

## Buttons and Events

### Event Handling

#### Buttons
Buttons are set up in correspondence with the Event Handling, each button is given a specific variable to prevent continous input, and will switch off once it has been released.  

In [ ]:
if button_a.is_pressed():
    if not aAct:
        aAct = True
        eventHandList.append("A")
else:
    if aAct:
        aAct = False

The button, when activated adds their identifier to the eventHandList to be processed, in this case we appended "A" to the Event Handle List.

##### Ports / Gold Icon
These are sightly different due to them having the ability to be active and deactived several times even though the user intended to only give one event. This is due to the sensors not being reliable and isn't a simple on/off input.

In [ ]:
if pin_logo.is_touched():
    if not gAct:
        gAct = True
        eventHandList.append("G")
else:
    if gAct:
        gAct = False

Since this code still gives multiple inputs, in our Event Handling we have to remove all extra instances of these types of inputs.

In [ ]:
if eventHandList[0] == "G" or eventHandList[0] == "P1":
    e = eventHandList[0]
    clrR = True
    while clrR:
        if len(eventHandList) > 1:
            if eventHandList[1] == e:
                eventHandList.pop(1)
            else:
                clrR = False
        else:
            clrR = False

This code will first check if the first item of the eventHandList is either the gold button event or a Port 1 event

`eventHandList = ["G", "G", "G", "A"]`

Then it is going to set a while active loop with the variable "clrR". Then inside the loop if the length of the Event List is greater than 0 (if there isn't any other events afterwards). It checks if the second index (1) is also the same event, and if so it will delete that index, else if its not the same event it will stop the loop. Lastly if the length of the event handler is only 1 indext long it will also stop the loop for the event to be further processed.

`eventHandList = ["G","A"]`

#### Event Handler


##### Input / Event Handling
These are the conditions for the events, inputs, outputs, and others.

In [ ]:
if len(eventHandList) > 0:
    nHrt = True
    while nHrt:
        if len(eventHandList) > 0:
            if eventHandList[0] == "G" or eventHandList[0] == "P1":

                #Clear extra port events
                e = eventHandList[0]
                clrR = True
                while clrR:
                    if len(eventHandList) > 1:
                        if eventHandList[1] == e:
                            eventHandList.pop(1)
                        else:
                            clrR = False
                    else:
                        clrR = False

                # Menu Inputs
                if menuA:

                    # Menu, Gold Input
                    if eventHandList[0] == "G":
                        tabA = getattr(globalValues,"tabA")

                        # Selected Tab is Settings
                        if tabA == 0:
                            menuA = False
                            settA = True
                            sleep(100)
                            display.clear()
                            settI()

                        # Selected Tab is Pong
                        if tabA == 1:
                            menuA = False
                            pongA = True
                            sleep(100)
                            display.clear()
                            pongI()
                else:

                    # Any other program presses the Gold Input. Go to Menu.
                    if eventHandList[0] == "G":
                        pongA = False
                        settA = False
                        menuA = True
                        sleep(100)
                        display.clear()
                        menuI()
                    elif eventHandList[0] == "P1":
                        if settA:
                            settP1T()
                eventHandList.pop(0)
            else:

                # Button Event Inputs
                if eventHandList[0] == "A" or eventHandList[0] == "B":
                    if eventHandList[0] == "A":

                    # If process is active, fire corresponding process for the event.
                        if pongA:
                            pongABtn()
                        elif menuA:
                            menuABtn()
                        elif settA:
                            settABtn()

                    elif eventHandList[0] == "B":
                        if pongA:
                            pongBBtn()
                        elif menuA:
                            menuBBtn()
                        elif settA:
                            settBBtn()
                    eventHandList.pop(0)
                else:
                    #Hearbeat Call Action, Custom events that are on timers 
                    if pongA:
                        pingHrt()

                    # All front non-heartbeat events are cleared.
                    nHrt = False
        else:
            nHrt = False

First, if there is any events in the event list, continue, Then it will create a while loop variable and clear out all inputs **instantanously**, where as custom Heartbeat events are handled differently. This is done incase it is needed for *sleeping during hearbeat* functions and to clear out input events faster than the heartbeat ones.

We then clear out extra Pin / Gold events, this is further broken down in [this section](#ports--gold-icon). Next the A and B buttons are handled next, I gave each process their own functions to handle each input. After each branch of conditions we then delete the first event to move onto the next.

```
pongABtn()
pongBBtn()
```

If the event is not "A" or "B" or "G" or "P1", then it has to be a custom hearbeat event. Hearbeat Call Action, will fire if the process is active and the function `pingHrt()`, will be fired. And finally cleaned up by setting notHeart (nHrt) to false.

##### Event List
The event list is stated at the beginning of the code, in order to maintain scope. The variable is named eventHandList, and is an array.  

`eventHandList = []`

After each input it will append to the end of this list to be processed.